In [1]:
# Necessary impports and pip installs
!pip3 install googlesearch-python
import pandas as pd
import os
import numpy as np
import re
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import zipfile
import time

In [2]:
'''
Dataframe loading. The initial dataframes are zipped
big_drone_df : large raw FAA drone intrusion info.
airport_codes_2 : huge dataset with info on airport codes, latitude & longitude
'''

zip_file_path_drone = 'combined_UAS_sightings.csv.zip'
csv_file_name_drone = 'combined_UAS_sightings.csv'  

with zipfile.ZipFile(zip_file_path_drone) as z:
    with z.open(csv_file_name_drone) as f:
        big_drone_df = pd.read_csv(f)

zip_file_path_airport = "all-airport-data.xlsx.zip"
xlsx_file_name_airport = "all-airport-data.xlsx"

with zipfile.ZipFile(zip_file_path_airport) as z:
    with z.open(xlsx_file_name_airport) as f:
        airport_codes_2 = pd.read_excel(f)

/Users/vishalivallioor/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
excel_df = pd.read_excel("fy24_q2.xlsx")
excel_df["Source File"] = "fy25_q1.xlsx"

In [4]:
big_df = pd.concat([big_drone_df, excel_df], ignore_index=True)
big_drone_df = big_df

In [5]:
big_drone_df = big_drone_df[~big_drone_df['Summary'].str.startswith('_x000D', na=False)]

# Reset index if needed (optional)
big_drone_df.reset_index(drop=True, inplace=True)
big_drone_df = big_drone_df.dropna(subset=['Summary'])

In [6]:
''''
Line of code to extract all the 3 letter airport codes. 
'''

list_codes = list(airport_codes_2["Loc Id"])

big_drone_df['code'] = None

for index_2, row in big_drone_df.iterrows():
  x = row["Summary"]
  splitted_x = x.split()
  punc = '''!()-[]{};:'"\,<>.,/?@#$%^&*_~'''

  # Removing punctuations in string
  # Using loop + punctuation string
  index = 0
  for word in splitted_x:
      for letter in word:
        if letter in punc:
          word = word.replace(letter, "")
          splitted_x[index] = word
      index = index + 1
  filtered_codes = list(set([i for i in splitted_x if len(i) == 3 and i not in ["FAA", "UAS", "OPS", "THE"]]))
  for i in filtered_codes:
    if i in list_codes:
       big_drone_df.at[index_2, 'code'] = i

big_drone_df

,Date,State,City,Summary,Source File,code
0,4/1/2022,PENNSYLVANIA,PITTSBURGH,"PRELIM INFO FROM FAA OPS: PITTSBURGH, PA/UAS I...",Reported-UAS-Sightings-April-June-2022.xlsx,AGC
1,4/1/2022,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",Reported-UAS-Sightings-April-June-2022.xlsx,DAL
2,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",Reported-UAS-Sightings-April-June-2022.xlsx,BFI
3,4/1/2022,ALABAMA,MOBILE,"PRELIM INFO FROM FAA OPS: MOBILE, AL/UAS INCID...",Reported-UAS-Sightings-April-June-2022.xlsx,FMY
4,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",Reported-UAS-Sightings-April-June-2022.xlsx,SEA
...,...,...,...,...,...,...
18215,2024-03-15 00:00:00,PR,MAYAGUEZ,"PRELIM INFO FROM FAA OPS: MAYAGUEZ, PUERTO RIC...",fy25_q1.xlsx,SJU
18216,2024-03-31 00:00:00,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",fy25_q1.xlsx,ATL
18217,2024-03-31 00:00:00,CALIFORNIA,FRESNO,"PRELIM INFO FROM FAA OPS: FRESNO, CA/UAS SIGHT...",fy25_q1.xlsx,CCR
18218,2024-03-31 00:00:00,NEW JERSEY,CALDWELL,"PRELIM INFO FROM FAA OPS: CALDWELL, NJ/UAS SIG...",fy25_q1.xlsx,CDW


In [7]:
drone_df = big_drone_df.copy()

In [8]:
'''
All regex commands for drone information extraction.
'''

def extract_direction(summary):
    match = re.search(r'WHILE\s([A-Z]+\sBOUND)', summary)
    if match:
        return match.group(1)
    else:
        return np.nan

drone_df['Direction'] = drone_df['Summary'].apply(extract_direction)

pattern_time = r'(?<=\/)([0-9]{4}[A-Z])'
drone_df['Time'] = drone_df['Summary'].str.extract(pattern_time)

pattern_advised = r'(?<=\/)([A-Z ]+)(?= ADVISED)'
drone_df['Authority Advised'] = drone_df['Summary'].str.extract(pattern_advised)

pattern_route = r'(?<=, )([A-Z  ,()]+ - [A-Z  ,()]+)(?=,)'
drone_df['Route'] = drone_df['Summary'].str.extract(pattern_route)

pattern_location = r'(?<= )([0-9]+ [(W|ESE|E|N|NW|SW|S|SE|NE|NNE|NNW|WNW|WSW|NORTHWEST|ENE|SSE|SSW)]+ [A-Z() ]+)(?= AT| FEET|[.])'
drone_df['Location'] = drone_df['Summary'].str.extract(pattern_location)

pattern_altitude = r'(?<= )(AT [0-9,]+ FEET)(?= )'
drone_df['Altitude'] = drone_df['Summary'].str.extract(pattern_altitude)

drone_df

,Date,State,City,Summary,Source File,code,Direction,Time,Authority Advised,Route,Location,Altitude
0,4/1/2022,PENNSYLVANIA,PITTSBURGH,"PRELIM INFO FROM FAA OPS: PITTSBURGH, PA/UAS I...",Reported-UAS-Sightings-April-June-2022.xlsx,AGC,S BOUND,2358E,ALLEGHENY COUNTY ATCT,NaN,6 N AGC,"AT 1,600 FEET"
1,4/1/2022,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",Reported-UAS-Sightings-April-June-2022.xlsx,DAL,W BOUND,1205E,NaN,DAL - OPF,5 E OPF,"AT 1,500 FEET"
2,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",Reported-UAS-Sightings-April-June-2022.xlsx,BFI,NNE BOUND,1210P,SEATTLE TRACON,HDN - BFI,15 WNW SEATTLE,"AT 3,000 FEET"
3,4/1/2022,ALABAMA,MOBILE,"PRELIM INFO FROM FAA OPS: MOBILE, AL/UAS INCID...",Reported-UAS-Sightings-April-June-2022.xlsx,FMY,SE BOUND,1859C,MOBILE APCH,BFM - FMY,14 N SEMMES VOR,"AT 6,000 FEET"
4,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",Reported-UAS-Sightings-April-June-2022.xlsx,SEA,W BOUND,1805P,SEATTLE TRACON,"KEFLAVIK, ICELAND (BIKF) - SEA",17 N SEA,"AT 7,000 FEET"
...,...,...,...,...,...,...,...,...,...,...,...,...
18215,2024-03-15 00:00:00,PR,MAYAGUEZ,"PRELIM INFO FROM FAA OPS: MAYAGUEZ, PUERTO RIC...",fy25_q1.xlsx,SJU,E BOUND,1143L,SAN JUAN CERAP,"LA ROMANA, DOMINICAN REPUBLIC (MDLR) - SJU",10 S MAYAGUEZ (TJMZ),"AT 9,000 FEET"
18216,2024-03-31 00:00:00,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",fy25_q1.xlsx,ATL,N BOUND,1210E,ATLANTA TRACON,MYR - ATL,20 SE ATL,"AT 5,000 FEET"
18217,2024-03-31 00:00:00,CALIFORNIA,FRESNO,"PRELIM INFO FROM FAA OPS: FRESNO, CA/UAS SIGHT...",fy25_q1.xlsx,CCR,SE BOUND,0920P,FRESNO ATCT,CCR - VIS,10 WNW FAT,"AT 8,000 FEET"
18218,2024-03-31 00:00:00,NEW JERSEY,CALDWELL,"PRELIM INFO FROM FAA OPS: CALDWELL, NJ/UAS SIG...",fy25_q1.xlsx,CDW,NaN,1156E,MCGUIRE APCH,CDW - UNKN,NaN,NaN


In [9]:
edge_df = drone_df[drone_df["code"].isna()]
edge_df_location = edge_df[~edge_df["Location"].isna()]
list(edge_df_location["Summary"])[1]

'PRELIM INFO FROM FAA OPS: LA JUNTA, CO/UAS INCIDENT/1730M/DENVER ARTCC ADVISED, MOONEY M20P, UNKN - UNKN, REPORTED BLACK UAS FROM THE LEFT SIDE 100 FEET ABOVE ACFT WHILE NNW BOUND AT 10,100 FEET 20 S LA JUNTA. NO EVASIVE ACTION TAKEN. LA JUNTA PD NOTIFIED. WOC 7-3333 ER/TB'

In [10]:
edge_df_location['City Extracted'] = edge_df_location['Location'].str.extract(r'(?:\d+\s+)?(?:NORTH|SOUTH|EAST|WEST|NE|NW|SE|SW|N|S|E|W)\s+(.+)$', expand=False)

# If there was no direction (city name already correct), retain original value
edge_df_location['City Extracted'] = edge_df_location['City Extracted'].fillna(edge_df_location['Location'])

# Trim any extra whitespace
edge_df_location['City Extracted'] = edge_df_location['City Extracted'].str.strip()

# Replace cases where extraction failed (e.g., "13 SE" with no city) with NaN
edge_df_location['City Extracted'] = edge_df_location['City Extracted'].where(edge_df_location['City Extracted'].str.len() > 1)

/var/folders/97/l1_w165s1z38tw82rlydjzp00000gn/T/ipykernel_54505/3281533863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edge_df_location['City Extracted'] = edge_df_location['Location'].str.extract(r'(?:\d+\s+)?(?:NORTH|SOUTH|EAST|WEST|NE|NW|SE|SW|N|S|E|W)\s+(.+)$', expand=False)
/var/folders/97/l1_w165s1z38tw82rlydjzp00000gn/T/ipykernel_54505/3281533863.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edge_df_location['City Extracted'] = edge_df_location['City Extracted'].fillna(edge_df_location

In [11]:
edge_df_location

,Date,State,City,Summary,Source File,code,Direction,Time,Authority Advised,Route,Location,Altitude,City Extracted
18,4/3/2022,OHIO,CLEVELAND,"PRELIM INFO FROM FAA OPS: CLEVELAND, OH/UAS IN...",Reported-UAS-Sightings-April-June-2022.xlsx,None,NW BOUND,1838E,CLEVELAND TRACON,NaN,13 SE CLEVELAND,"AT 7,200 FEET",CLEVELAND
33,4/6/2022,COLORADO,LA JUNTA,"PRELIM INFO FROM FAA OPS: LA JUNTA, CO/UAS INC...",Reported-UAS-Sightings-April-June-2022.xlsx,None,NNW BOUND,1730M,DENVER ARTCC,UNKN - UNKN,20 S LA JUNTA,"AT 10,100 FEET",LA JUNTA
52,4/10/2022,INDIANA,INDIANAPOLIS,"PRELIM INFO FROM FAA OPS: INDIANAPOLIS, IN/UAS...",Reported-UAS-Sightings-April-June-2022.xlsx,None,E BOUND,1105E,INDIANAPOLIS ATCT,NaN,6 ESE BRICKYARD VORTAC,"AT 1,600 FEET",BRICKYARD VORTAC
64,4/12/2022,ARIZONA,BUCKEYE,"PRELIM INFO FROM FAA OPS: BUCKEYE, AZ/UAS INCI...",Reported-UAS-Sightings-April-June-2022.xlsx,None,WNW BOUND,0831M,ALBUQUERQUE ARTCC,NaN,24 S BUCKEYE,"AT 17,000 FEET",BUCKEYE
88,4/17/2022,CALIFORNIA,MONTEREY,"PRELIM INFO FROM FAA OPS: MONTEREY, CA/UAS INC...",Reported-UAS-Sightings-April-June-2022.xlsx,None,E BOUND,0745P,MONTEREY ATCT,NaN,2 W MONTEREY,"AT 1,000 FEET",MONTEREY
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18066,2024-02-23 00:00:00,CALIFORNIA,POMONA,"PRELIM INFO FROM FAA OPS: POMONA, CA/UAS SIGHT...",fy25_q1.xlsx,None,W BOUND,2052E,SOUTHERN CALIFORNIA TRACON,NaN,5 W POMONA VORTAC (POM),"AT 3,500 FEET",POMONA VORTAC (POM)
18075,2024-02-24 00:00:00,TEXAS,GALVESTON,"PRELIM INFO FROM FAA OPS: GALVESTON, TX/UAS SI...",fy25_q1.xlsx,None,SE BOUND,1708C,HOUSTON TRACON,NaN,8 N SCHOLES VOR (VUH),"AT 3,000 FEET",SCHOLES VOR (VUH)
18124,2024-03-10 00:00:00,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",fy25_q1.xlsx,None,W BOUND,1600E,ATLANTA TRACON,NaN,18 E ATLANTA,"AT 3,000 FEET",ATLANTA
18162,2024-03-18 00:00:00,NEW YORK,DEER PARK,"PRELIM INFO FROM FAA OPS: DEER PARK, NY/UAS SI...",fy25_q1.xlsx,None,NW BOUND,1635E,REPUBLIC ATCT,NaN,5 SW DEER PARK VOR (DPK),AT 400 FEET,DEER PARK VOR (DPK)


In [12]:
'''
Using Beautiful Soup and Google Search with Airnav in order to map the values for which the FAA code could not be found in original dataset.
'''

drone_df['latitude'] = None
drone_df['longitude'] = None

# Iterate over rows
for idx, location in edge_df_location.iterrows():
    loc_list = location['City Extracted']
    full_string = " " + loc_list
    query_plain = " airnav site:airnav.com"
    query = full_string + query_plain

    # Perform Google search 
    search_results = search(query, num_results=1)  
    final_url = next(search_results, None)
    
    if final_url:
        headers = {
            "User-Agent": "your bot 0.1"
        }

        response = requests.get(final_url, headers=headers)

        if response.status_code == 200:
            print("\n")
            print("yes this one --")
            soup = BeautifulSoup(response.content, 'html.parser')
            
            faa_code = None
            latitude = None
            longitude = None
            
            table_rows = soup.find_all('tr')  
            for row in table_rows:
                if 'FAA Identifier' in row.text:
                    print(faa_code)
                    faa_code = row.find_all('td')[1].text.strip()  
                
                if 'Lat/Long' in row.text:
                    print(lat_long_text)
                    lat_long_text = row.find_all('td')[1].text.strip()
                    lat_long_split = lat_long_text.split("W")
                    
                    if len(lat_long_split) >= 2:
                        latitude = lat_long_split[0] + 'W' 
                        longitude = 'W' + lat_long_split[1]

            if not (latitude and longitude):
                string_soup = str(soup)
                result_re = re.findall(r'(?<=Lat\/Long:\s)([0-9\-\.]+[N|S])\s\/\s([0-9\-\.]+[E|W])', string_soup)
                if result_re:
                    latitude, longitude = result_re[0]

            if faa_code:
                drone_df.at[idx, 'code'] = faa_code
            if latitude and longitude:
                drone_df.at[idx, 'latitude'] = latitude
                drone_df.at[idx, 'longitude'] = longitude
        else:
            print(f"Failed to fetch the page for {location['Location']}, status code: {response.status_code}")

KeyboardInterrupt: 

In [17]:
query = ' ABILENE airnav site:airnav.com'

search_results = search(query, num_results=1)  
search_results

' ABILENE airnav site:airnav.com'

In [18]:
search_results = search(query, num_results=1)  
search_results

<generator object search at 0x1772552a0>

In [19]:
next(search_results)

HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3D%2BABILENE%2Bairnav%2Bsite%253Aairnav.com%26num%3D3%26hl%3Den%26start%3D0%26safe%3Dactive&hl=en&q=EhAgAQWoRTSDAACNTTMcIcesGIbL_70GIjA62Ue21Hw52SrPp8_TJ0JkrDPmb3s9EX1k_VRAntCDaOU6ZX1yBF4mJOo8cMD3S5kyAXJaAUM

StopIteration: 

In [9]:
'''
Merging the results with the initial airport codes dataframe to get latitude and longitude for all rows.
'''

merged_drone_df = pd.merge(drone_df, airport_codes_2[['Loc Id', 'ARP Latitude DD', 'ARP Longitude DD']], 
                           left_on='code', right_on='Loc Id', how='left')

# Step 2: Only update 'latitude' and 'longitude' where they are NaN in drone_df
merged_drone_df['latitude'] = merged_drone_df['latitude'].fillna(merged_drone_df['ARP Latitude DD'])
merged_drone_df['longitude'] = merged_drone_df['longitude'].fillna(merged_drone_df['ARP Longitude DD'])

# Optional: Drop the 'ARP Latitude DD', 'ARP Longitude DD', and 'Loc Id' columns if you don't need them
merged_drone_df.drop(['Loc Id', 'ARP Latitude DD', 'ARP Longitude DD'], axis=1, inplace=True)
merged_drone_df

,Date,State,City,Summary,Source File,code,Direction,Time,Authority Advised,Route,Location,Altitude,latitude,longitude
0,4/1/2022,PENNSYLVANIA,PITTSBURGH,"PRELIM INFO FROM FAA OPS: PITTSBURGH, PA/UAS I...",Reported-UAS-Sightings-April-June-2022.xlsx,AGC,S BOUND,2358E,ALLEGHENY COUNTY ATCT,NaN,6 N AGC,"AT 1,600 FEET",40.354438,-79.929047
1,4/1/2022,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",Reported-UAS-Sightings-April-June-2022.xlsx,DAL,W BOUND,1205E,NaN,DAL - OPF,5 E OPF,"AT 1,500 FEET",32.845945,-96.850877
2,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",Reported-UAS-Sightings-April-June-2022.xlsx,HDN,NNE BOUND,1210P,SEATTLE TRACON,HDN - BFI,15 WNW SEATTLE,"AT 3,000 FEET",40.481194,-107.217667
3,4/1/2022,ALABAMA,MOBILE,"PRELIM INFO FROM FAA OPS: MOBILE, AL/UAS INCID...",Reported-UAS-Sightings-April-June-2022.xlsx,FMY,SE BOUND,1859C,MOBILE APCH,BFM - FMY,14 N SEMMES VOR,"AT 6,000 FEET",26.586615,-81.863247
4,4/1/2022,WASHINGTON,SEATTLE,"PRELIM INFO FROM FAA OPS: SEATTLE, WA/UAS INCI...",Reported-UAS-Sightings-April-June-2022.xlsx,SEA,W BOUND,1805P,SEATTLE TRACON,"KEFLAVIK, ICELAND (BIKF) - SEA",17 N SEA,"AT 7,000 FEET",47.449889,-122.311778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17887,2019-12-31 00:00:00,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS INCIDE...",FY2020_Q1_UAS_Sightings.xlsx,TMB,W BOUND,1811E,MIAMI EXECUTIVE ATCT,NaN,2 WSW TMB,"AT 1,000 FEET",25.647564,-80.433225
17888,2019-12-31 00:00:00,FLORIDA,WEST PALM BEACH,"PRELIM INFO FROM FAA OPS: WEST PALM BEACH, FL/...",FY2020_Q1_UAS_Sightings.xlsx,DFW,E BOUND,1800E,PALM BEACH TRACON,DFW - PBI,1 W PALM BEACH,AT 400 FEET,32.897233,-97.037695
17889,2019-12-31 00:00:00,FLORIDA,FORT MYERS,"PRELIM INFO FROM FAA OPS: FORT MYERS, FL/UAS I...",FY2020_Q1_UAS_Sightings.xlsx,APF,S BOUND,1655E,FORT MYERS APCH,JVY - APF,8 SE FORT MYERS,"AT 7,000 FEET",26.152441,-81.775639
17890,2019-12-31 00:00:00,HAWAII,HONOLULU,"PRELIM INFO FROM FAA OPS: HONOLULU, HI/UAS INC...",FY2020_Q1_UAS_Sightings.xlsx,HNL,NaN,1512L,NaN,"HNL - LAHAINA, HI (PHJH)",NaN,"AT 1,000 FEET",21.317825,-157.92025


In [11]:
merged_drone_df.to_csv("fully_merged_dataset.csv")

In [10]:
''' 
EXTRACTING 3D COORDINATE OF DRONE BASED ON ALTITUDE, LONGITUDE, AND LATITUDE
Reference: 
    https://stackoverflow.com/questions/16614057/longitude-latitude-altitude-to-3d-cartesian-coordinate-systems
    for calculation of 3D coordinate
'''

# converting latitude and longitude columns to numeric values (currently an object)
merged_drone_df['latitude'] = pd.to_numeric(merged_drone_df['latitude'])
merged_drone_df['longitude'] = pd.to_numeric(merged_drone_df['longitude'])

# dropping rows with NaN values in latitude, longitude, or altitude
merged_drone_df = merged_drone_df.dropna(subset=['latitude', 'longitude', 'Altitude'])

# adjust altitude column with regex to delete the "AT" and "FEET" and only displays numbers
# convert the feet to meters since axis is in meters
merged_drone_df['altitude_numeric'] = merged_drone_df['Altitude'].str.extract(r'(\d{1,3}(?:,\d{3})*)')
merged_drone_df['altitude_numeric'] = merged_drone_df['altitude_numeric'].str.replace(',', '').astype(float)
merged_drone_df['alt_meters'] = merged_drone_df['altitude_numeric'] * 0.3048

# 3D coordinate extracting:

axis = 6378137
eccentricity = 8.1819190842622e-2
asq = np.power(axis, 2)
esq = np.power(eccentricity, 2)

def convert(df):
    df['lat_radian'] = np.radians(df['latitude'])
    df['long_radian'] = np.radians(df['longitude'])

    df['N'] = axis / np.sqrt(1 - esq * np.power(np.sin(df['lat_radian']), 2))

    df['x'] = (df['N'] + df['alt_meters']) * np.cos(df['lat_radian']) * np.cos(df['long_radian'])
    df['y'] = (df['N'] + df['alt_meters']) * np.cos(df['lat_radian']) * np.sin(df['long_radian'])
    df['z'] = ((1 - esq) * df['N'] + df['alt_meters']) * np.sin(df['lat_radian'])

    df.drop(columns=['lat_radian', 'long_radian', 'N'], inplace=True)

    return df

merged_drone_df2 = convert(merged_drone_df)
merged_drone_df2.head()

ValueError: Unable to parse string "41-24-33.8650N 081-51-16.8880W" at position 18